<i>Copyright (c) Recommenders contributors.</i>

<i>Licensed under the MIT License.</i>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data split

데이터 분할은 추천 시스템을 평가하는 데 있어 가장 중요한 작업 중 하나입니다. 분할 전략은 평가 프로토콜에 큰 영향을 미치므로 실무자가 항상 신중하게 고려해야 합니다.

아래 코드에서는 특정 시나리오에 대해 다양한 분할 전략을 적용하는 방법을 설명합니다.

## 0 Global settings

In [11]:
import pandas as pd
from datetime import datetime, timedelta
import sys
sys.path.append('/content/drive/Othercomputers/내 노트북/study/10_RecSystem/실습-20230923')

from msr.python_splitters import (
    python_random_split,
    python_chrono_split,
    python_stratified_split
)

In [12]:
COL_USER = "userId"
COL_ITEM = "movieId"
COL_RATING = "rating"
COL_TIMESTAMP = "timestamp"

## 1 Data preparation

### 1.1 Data understanding

설명을 위해 아래 예제에서 사용된 데이터는 MovieLens 데이터 세트입니다.

In [13]:
data = pd.read_csv("/content/drive/Othercomputers/내 노트북/study/10_RecSystem/실습-20230923/ratings.csv")

데이터 살펴보기

In [14]:
data.head()

,userId,movieId,rating,timestamp
0,1,0,4.0,964982703
1,1,2,4.0,964981247
2,1,5,4.0,964982224
3,1,43,5.0,964983815
4,1,46,5.0,964982931


A little more...

In [15]:
data.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,3101.735561,3.501557,1.205946e+09
std,182.618491,2627.050983,1.042529,2.162610e+08
min,1.000000,0.000000,0.500000,8.281246e+08
25%,177.000000,900.000000,3.000000,1.019124e+09
50%,325.000000,2252.000000,3.500000,1.186087e+09
75%,477.000000,5095.250000,4.000000,1.435994e+09
max,610.000000,9723.000000,5.000000,1.537799e+09


And, more...

In [16]:
print(
    "Total number of ratings are\t{}".format(data.shape[0]),
    "Total number of users are\t{}".format(data[COL_USER].nunique()),
    "Total number of items are\t{}".format(data[COL_ITEM].nunique()),
    sep="\n"
)

Total number of ratings are	100836
Total number of users are	610
Total number of items are	9724


### 1.2 Data transformation

원본 타임스탬프를 ISO 형식으로 변환

In [17]:
data[COL_TIMESTAMP]= data.apply(
    lambda x: datetime.strftime(datetime(1970, 1, 1, 0, 0, 0) + timedelta(seconds=x[COL_TIMESTAMP].item()), "%Y-%m-%d %H:%M:%S"),
    axis=1
)

In [18]:
data.head()

,userId,movieId,rating,timestamp
0,1,0,4.0,2000-07-30 18:45:03
1,1,2,4.0,2000-07-30 18:20:47
2,1,5,4.0,2000-07-30 18:37:04
3,1,43,5.0,2000-07-30 19:03:35
4,1,46,5.0,2000-07-30 18:48:51


## 2 Experimentation protocol

실험 프로토콜은 일반적으로 특정 추천 시나리오에 대한 합리적인 평가에 유리하도록 설정됩니다. 예를 들어,
* *Recommender-A*는 사람들의 공동 평가 유사도를 취하여 사람들에게 영화를 추천하는 것입니다. 평가가 통계적으로 의미있는지 확인하기 위해 모델 구축과 테스트 모두에 동일한 사용자 집합을 사용해야 하며(사용자 cold-start를 피하기 위해) 계층화된 분할 전략을 취해야 합니다.
* *Recommender-B*는 고객에게 패션 상품을 추천하는 것입니다. 패션 상품에 대한 고객의 취향은 시간이 지남에 따라 변할 수 있기 때문에 추천시스템의 평가는 고객 구매의 시간 의존성을 고려하는 것이 합리적입니다. 이 경우 시간대별 분할을 사용해야 합니다.

## 3 Data split

### 3.1 Random split (무작위 분할)

무작위 분할은 단순히 데이터 집합을 가져와 분할 비율에 따라 데이터의 분할을 출력합니다.

In [19]:
data_train, data_test = python_random_split(data, ratio=0.9)

In [20]:
data_train.shape[0], data_test.shape[0]

(90752, 10084)

In [21]:
# 학습 데이터에는 있는지만 평가 데이터에는 없는 사용자 확인
set(data_train.userId) - set(data_test.userId)

{138, 158, 257, 375, 392, 496, 578}

때로는 다중 분할(2개 이상)이 필요한 경우도 있습니다.

In [22]:
data_train, data_validate, data_test = python_random_split(data, ratio=[0.6, 0.2, 0.2])

In [23]:
data_train.shape[0], data_validate.shape[0], data_test.shape[0]

(60502, 20167, 20167)

비율은 정수일 수도 있습니다.

In [24]:
data_train, data_validate, data_test = python_random_split(data, ratio=[3, 1, 1])

동일한 결과를 생성합니다.

In [25]:
data_train.shape[0], data_validate.shape[0], data_test.shape[0]

(60502, 20167, 20167)

### 3.2 Chronological split (시간순 분할)

시간순 분할 방법은 데이터 집합을 가져와서 타임스탬프를 기준으로 분할합니다.

#### 3.2.1 "Filter by"

시간순 분할은 '사용자' 또는 '아이템'을 기준으로 할 수 있습니다. 예를 들어, '사용자' 기준이고 분할 비율이 0.7인 경우, 데이터에서 각 사용자의 첫 70%의 평가가 한 분할에 포함되고 나머지 30%는 다른 분할에 포함된다는 의미입니다. 시간순 분할은 타임스탬프에 따라 달라지므로 시간순 분할은 "무작위"가 아니라는 점에 유의할 필요가 있습니다.

In [26]:
data_train, data_test = python_chrono_split(
    data, ratio=0.7, filter_by="user",
    col_user=COL_USER, col_item=COL_ITEM, col_timestamp=COL_TIMESTAMP
)

특정 사용자 한 명에 대한 결과를 살펴보세요:

* 학습 데이터의 마지막 10행:

In [27]:
data_train[data_train[COL_USER] == 1].tail(10)

,userId,movieId,rating,timestamp
56,1,789,5.0,2000-07-30 18:46:31
138,1,1598,5.0,2000-07-30 18:47:18
127,1,1542,5.0,2000-07-30 18:47:18
38,1,551,5.0,2000-07-30 18:47:18
131,1,1558,4.0,2000-07-30 18:47:18
35,1,513,5.0,2000-07-30 18:47:18
128,1,1552,5.0,2000-07-30 18:47:18
53,1,786,5.0,2000-07-30 18:47:35
51,1,782,5.0,2000-07-30 18:47:56
135,1,1576,5.0,2000-07-30 18:47:56


* 평가 데이터의 처음 10행:

In [28]:
data_test[data_test[COL_USER] == 1].head(10)

,userId,movieId,rating,timestamp
230,1,2987,4.0,2000-07-30 18:48:23
122,1,1504,5.0,2000-07-30 18:48:23
54,1,787,3.0,2000-07-30 18:48:23
4,1,46,5.0,2000-07-30 18:48:51
36,1,520,5.0,2000-07-30 18:48:51
74,1,913,5.0,2000-07-30 18:49:11
103,1,1217,5.0,2000-07-30 18:49:11
62,1,827,5.0,2000-07-30 18:49:11
16,1,257,3.0,2000-07-30 18:49:27
144,1,1686,4.0,2000-07-30 18:49:49


훈련 데이터의 타임스탬프는 모두 평가 데이터의 타임스탬프보다 앞선 것입니다.

In [29]:
# 학습 데이터에는 있는지만 평가 데이터에는 없는 사용자 확인
set(data_train.userId) - set(data_test.userId)

set()

### 3.3 Stratified split (계층적 분할)

계층적 분할은 계층화되어 동일한 사용자 또는 항목 세트가 학습 및 테스트 데이터 세트에 모두 나타납니다.

In [30]:
data_train, data_test = python_stratified_split(
    data, filter_by="user", ratio=0.7,
    col_user=COL_USER, col_item=COL_ITEM
)

In [31]:
# 학습 데이터에는 있는지만 평가 데이터에는 없는 사용자 확인
set(data_train.userId) - set(data_test.userId)

set()

# End